<a href="https://colab.research.google.com/github/YugaYoshimura/a/blob/main/Sotsuron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Chromiumとseleniumをインストール
print("前処理を開始")
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
     

前処理を開始
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.4 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad

In [2]:
import time
import json
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [3]:
pd.options.display.max_rows = 200

# アカウント情報の読込
account = pd.read_csv("account.txt", header=None, sep=',')
MAIL = account.loc[0,0]
PASS = account.loc[0,1]

In [4]:
def login_fun(_mail, _pass):
    # ログインページにアクセス
    driver.get("https://m-league.jp/supporter/login")
    
    # Mail, Passを入力
    #id_input = driver.find_element_by_id("login_mail")
    id_input = driver.find_element(By.ID, "login_mail")
    id_input.send_keys(_mail)
    #pass_input = driver.find_element_by_id("login_password")
    pass_input = driver.find_element(By.ID, "login_password")
    pass_input.send_keys(_pass)
    
    # ログインボタンをクリック
    #login_button = driver.find_elements_by_name("login")
    #login_button = driver.find_element(By.ID, "login")
    #login_button[1].click()
    button1_xpath = '//*[@id="app"]/div/main/section/div/section[1]/div/button' 
    button1 = driver.find_element(By.XPATH,button1_xpath)
    button1.click()

def get_match_schedule_fun(season):
    
    match_schedule_list = []
    
    # 試合日程ページへアクセス
    if season == 2022:
        url = "https://m-league.jp/games"
    else:
        url = f"https://m-league.jp/games/{season}-season"
    driver.get(url)
    
    # 20xxシーズンは20xx+1まで開催
    for y in [season, season+1]:
        for m in range(1, 12+1, 1):
                for d in range(1, 31+1, 1):
                    ymd = str(y) + str(m).zfill(2) + str(d).zfill(2)
                    if len(driver.find_elements(By.XPATH,f".//div[@id='js-modal-key{ymd}']")) == 1:
                        match_schedule_list += [ymd]
    
    return match_schedule_list

def make_score_data_fun(html):
    # htmlをパースする
    soup = BeautifulSoup(html, "html.parser")
    
    # 必要な個所だけ抽出
    start_mark = "UMP_PLAYER.init(true, true, \'"
    end_mark = "\', autoplay);"
    start_position = str(soup).find(start_mark) + len(start_mark)
    end_position = str(soup).find(end_mark)
    score = str(soup)[start_position:end_position]
    
    # DataFrameに変換
    dat = pd.DataFrame(json.loads(score))
    
    return dat

def make_dat_2021_fun(match_schedule_list_2021):
    driver.get("https://m-league.jp/games/2021-season")

    dat_list = []
    match_idx = 1
    # 試合日程は時系列にソート済み想定
    for m_s in match_schedule_list_2021:
        # 月選択
#         tmp = driver.find_element(By.XPATH,f".//a[@href='/games/?mly={m_s[0:4]}&mlm={int(m_s[4:6])}#schedule']")
        tmp = driver.find_element(By.CSS_SELECTOR, "#wrapper > div > main > section > div.p-gamesSchedule2 > div > ul:nth-child(3) > li:nth-child(2)")
        tmp2 = driver.find_elements(By.CSS_SELECTOR, "#wrapper > div > main > section > div.p-gamesSchedule2 > div > ul:nth-child(3)")
        for t in tmp2:
            print(t)
        tmp.click()
        # 日程選択
#         tmp = driver.find_element(By.XPATH,f".//li[@data-target='key{m_s}']")
        tmp = driver.find_element(By.CSS_SELECTOR,f"#js-modal-key{match_schedule_list_2021[match_idx]} > div.c-modal2__contents > div > div.p-gamesResult__columns > div:nth-child(1) > form > button")

        tmp.click()
        
        time.sleep(1)
        # 牌譜ビューアにアクセスするボタン取得
#         button = driver.find_elements(By.XPATH,'//div[@class="c-button -primary -small"]')
        button1 = driver.find_element(By.CSS_SELECTOR, "#js-modal-key20211004 > div.c-modal2__contents > div > div.p-gamesResult__columns > div:nth-child(1) > form > button")
        button2 = driver.find_element(By.CSS_SELECTOR, "#js-modal-key20211004 > div.c-modal2__contents > div > div.p-gamesResult__columns > div:nth-child(2) > form > button")
        button1.click()

        ### 第1試合
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[1])
        # 文字コードをUTF-8に変換
        html = driver.page_source.encode('utf-8')
        # DataFrameに変換
        dat = make_score_data_fun(html)
        dat_list += [dat]
        # 牌譜ビューアを閉じる
        driver.close()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[0])

        ### 第2試合
        button2.click()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[1])
        # 文字コードをUTF-8に変換
        html = driver.page_source.encode('utf-8')
        # DataFrameに変換
        dat = make_score_data_fun(html)
        dat_list += [dat]
        # 牌譜ビューアを閉じる
        driver.close()
        # 制御するタブの移動
        driver.switch_to.window(driver.window_handles[0])

        # ページのリセット
        driver.get("https://m-league.jp/games/2021-season")

        # アクセス制限
        time.sleep(10)
        match_idx += 1
        print("Get: " + m_s)
        
    # データ縦積み
    dat_all = pd.concat(dat_list).reset_index(drop=True)
    
    return dat_all
    

In [18]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')
     

In [16]:
from selenium.webdriver.common.by import By

In [20]:
# ブラウザのオプション
options = Options()

from selenium.webdriver.chrome.service import Service

#driver = webdriver.Chrome(service=Service('/usr/lib/chromium-browser/chromedriver'))

# ブラウザ起動
driver = webdriver.Chrome(executable_path="/Users/yugayoshimura/Downloads", options=options)

<ipython-input-20-785ad28fd36e>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/yugayoshimura/Downloads")


WebDriverException: ignored

In [8]:
login_fun(MAIL, PASS)

NameError: ignored

In [ ]:
# 2022-23シーズン
match_schedule_list_2022 = get_match_schedule_fun(2022)
match_schedule_list_2022.sort(reverse=False)
len(match_schedule_list_2021)

In [ ]:
## 2021-22シーズン
match_schedule_list_2021 = get_match_schedule_fun(2021)
match_schedule_list_2021.sort(reverse=False)
len(match_schedule_list_2021)

In [ ]:
#2020-21シーズン
match_schedule_list_2020 = get_match_schedule_fun(2020)
match_schedule_list_2020.sort(reverse=False)
len(match_schedule_list_2020)

In [ ]:
# 2019-20シーズン
match_schedule_list_2019 = get_match_schedule_fun(2019)
match_schedule_list_2019.sort(reverse=False)
len(match_schedule_list_2019)

In [ ]:
# 2018-19シーズン
match_schedule_list_2018 = get_match_schedule_fun(2018)
match_schedule_list_2018.sort(reverse=False)
len(match_schedule_list_2018)

In [ ]:

def login_fun(_mail, _pass):
    # ログインページにアクセス
    driver.get("https://m-league.jp/supporter/login")
    
    # Mail, Passを入力
    #id_input = driver.find_element_by_id("login_mail")
    id_input = driver.find_element(By.ID, "login_mail")
    id_input.send_keys(_mail)
    #pass_input = driver.find_element_by_id("login_password")
    pass_input = driver.find_element(By.ID, "login_password")
    pass_input.send_keys(_pass)
    
    # ログインボタンをクリック
    #login_button = driver.find_elements_by_name("login")
    #login_button = driver.find_element(By.ID, "login")
    #login_button[1].click()
    button1_xpath = '//*[@id="app"]/div/main/section/div/section[1]/div/button' 
    button1 = driver.find_element(By.XPATH,button1_xpath)
    button1.click()

def get_match_schedule_fun(season):
    
    match_schedule_list = []
    
    # 試合日程ページへアクセス
    if season == 2022:
        url = "https://m-league.jp/games"
    else:
        url = f"https://m-league.jp/games/{season}-season"
    driver.get(url)
    
    # 20xxシーズンは20xx+1まで開催
    for y in [season, season+1]:
        for m in range(1, 12+1, 1):
                for d in range(1, 31+1, 1):
                    ymd = str(y) + str(m).zfill(2) + str(d).zfill(2)
                    if len(driver.find_elements(By.XPATH,f".//div[@id='js-modal-key{ymd}']")) == 1:
                        match_schedule_list += [ymd]
    
    return match_schedule_list

def make_score_data_fun(html):
    # htmlをパースする
    soup = BeautifulSoup(html, "html.parser")
    
    # 必要な個所だけ抽出
    start_mark = "UMP_PLAYER.init(true, true, \'"
    end_mark = "\', autoplay);"
    start_position = str(soup).find(start_mark) + len(start_mark)
    end_position = str(soup).find(end_mark)
    score = str(soup)[start_position:end_position]
    
    # DataFrameに変換
    dat = pd.DataFrame(json.loads(score))
    
    return dat

def make_dat_2021_fun(match_schedule_list_2021):
    driver.get("https://m-league.jp/games/2021-season")

    dat_list = []
    
    month_idx = 2
    day_idx = 1
    match_idx = 0
    # 試合日程は時系列にソート済み想定
    while True:
        month_idx += 1
        #bottun_next = driver.find_element(By.CSS_SELECTOR, f"#wrapper > div > main > section > div.p-gamesSchedule2 > div > nav > ul > li:nth-child(2)")
        #bottun_next.click()
        
        try:
            check = driver.find_element(By.CSS_SELECTOR, f"#wrapper > div > main > section > div.p-gamesSchedule2 > div > ul:nth-child({month_idx}) > li:nth-child({day_idx})")
            print(f"#wrapper > div > main > section > div.p-gamesSchedule2 > div > ul:nth-child({month_idx}) > li:nth-child({day_idx})")
            while True:
                try:
                    print(f"month: {month_idx}, day: {day_idx}")
                    # 月選択
            #         tmp = driver.find_element(By.XPATH,f".//a[@href='/games/?mly={m_s[0:4]}&mlm={int(m_s[4:6])}#schedule']")
                    tmp = driver.find_element(By.CSS_SELECTOR, f"#wrapper > div > main > section > div.p-gamesSchedule2 > div > ul:nth-child({month_idx}) > li:nth-child({day_idx})")
                    tmp.click()
                    # 日程選択
            #         tmp = driver.find_element(By.XPATH,f".//li[@data-target='key{m_s}']")
                    print(match_schedule_list_2021[match_idx])
                    tmp = driver.find_element(By.CSS_SELECTOR,f"#js-modal-key{match_schedule_list_2021[match_idx]} > div.c-modal2__contents > div > div.p-gamesResult__columns > div:nth-child(1) > form > button")

                    tmp.click()

                    time.sleep(1)
                    # 牌譜ビューアにアクセスするボタン取得
                    button1 = driver.find_element(By.CSS_SELECTOR, f"#js-modal-key{match_schedule_list_2021[match_idx]} > div.c-modal2__contents > div > div.p-gamesResult__columns > div:nth-child(1) > form > button")
                    button2 = driver.find_element(By.CSS_SELECTOR, f"#js-modal-key{match_schedule_list_2021[match_idx]} > div.c-modal2__contents > div > div.p-gamesResult__columns > div:nth-child(1) > form > button")
                    button1.click()

                    ### 第1試合
                    # 制御するタブの移動
                    driver.switch_to.window(driver.window_handles[1])
                    # 文字コードをUTF-8に変換
                    html = driver.page_source.encode('utf-8')
                    # DataFrameに変換
                    dat = make_score_data_fun(html)
                    dat_list += [dat]
                    # 牌譜ビューアを閉じる
                    driver.close()
                    # 制御するタブの移動
                    driver.switch_to.window(driver.window_handles[0])

                    ### 第2試合
                    button2.click()
                    # 制御するタブの移動
                    driver.switch_to.window(driver.window_handles[1])
                    # 文字コードをUTF-8に変換
                    html = driver.page_source.encode('utf-8')
                    # DataFrameに変換
                    dat = make_score_data_fun(html)
                    dat_list += [dat]
                    # 牌譜ビューアを閉じる
                    driver.close()
                    # 制御するタブの移動
                    driver.switch_to.window(driver.window_handles[0])

                    # ページのリセット
                    driver.get("https://m-league.jp/games/2021-season")

                    # アクセス制限
                    time.sleep(10)
                    day_idx += 1
                    match_idx += 1
                except Exception as e:
                    day_idx = 1
                    print("１ヶ月終わり")
                    print(e)
                    bottun_next = driver.find_element(By.CSS_SELECTOR, f"#wrapper > div > main > section > div.p-gamesSchedule2 > div > nav > ul > li:nth-child({month_idx - 1})")
                    bottun_next.click()
                    break
                    
        except Exception as e:
            print(e)
            break
            
        
    # データ縦積み
    dat_all = pd.concat(dat_list).reset_index(drop=True)
    
    return dat_all

In [ ]:
dat_2021 = make_dat_2021_fun(match_schedule_list_2021)

In [ ]:
dat_2021.to_csv("../Downloads/M_League_2021.csv", header=False, index=False)

print(dat_2021)